In [1]:
using Oceananigans

In [2]:
using TopographicHorizontalConvection: HorizontalConvectionSimulation

In [3]:
simulation = HorizontalConvectionSimulation(Ra=1e9, h₀_frac=0.6, advection=true)

┌ Warning: Overwriting existing ./../output/turbulent_h0.6_buoyancy.nc.
└ @ Oceananigans.OutputWriters /Users/hfdrake/.julia/packages/Oceananigans/thcPW/src/OutputWriters/netcdf_output_writer.jl:433
┌ Warning: Overwriting existing ./../output/turbulent_h0.6_velocities.nc.
└ @ Oceananigans.OutputWriters /Users/hfdrake/.julia/packages/Oceananigans/thcPW/src/OutputWriters/netcdf_output_writer.jl:433
┌ Warning: Overwriting existing ./../output/turbulent_h0.6_zonal_time_means.nc.
└ @ Oceananigans.OutputWriters /Users/hfdrake/.julia/packages/Oceananigans/thcPW/src/OutputWriters/netcdf_output_writer.jl:433


Simulation of NonhydrostaticModel{CPU, ImmersedBoundaryGrid}(time = 0 seconds, iteration = 0)
├── Next time step: 17.678 ms
├── Elapsed wall time: 0 seconds
├── Wall time per iteration: NaN days
├── Stop time: 50 seconds
├── Stop iteration : Inf
├── Wall time limit: Inf
├── Callbacks: OrderedDict with 6 entries:
│   ├── stop_time_exceeded => Callback of stop_time_exceeded on IterationInterval(1)
│   ├── stop_iteration_exceeded => Callback of stop_iteration_exceeded on IterationInterval(1)
│   ├── wall_time_limit_exceeded => Callback of wall_time_limit_exceeded on IterationInterval(1)
│   ├── nan_checker => Callback of NaNChecker for u on IterationInterval(100)
│   ├── wizard => Callback of TimeStepWizard(cfl=0.5, max_Δt=Inf, min_Δt=0.0) on IterationInterval(50)
│   └── progress => Callback of (::TopographicHorizontalConvection.var"#progress#6") on IterationInterval(10)
├── Output writers: OrderedDict with 5 entries:
│   ├── checkpointer => Checkpointer{TimeInterval, Vector{Symbol}}
│  

In [5]:
run!(simulation, pickup=false)

┌ Info: Initializing simulation...
└ @ Oceananigans.Simulations /Users/hfdrake/.julia/packages/Oceananigans/thcPW/src/Simulations/run.jl:184


i:      0, sim time:  0.000, wall time:  0 seconds, Δt:  0.0194, advective CFL: 0.00e+00, diffusive CFL: 6.30e-04


┌ Info:     ... simulation initialization complete (13.004 seconds)
└ @ Oceananigans.Simulations /Users/hfdrake/.julia/packages/Oceananigans/thcPW/src/Simulations/run.jl:220
┌ Info: Executing initial time step...
└ @ Oceananigans.Simulations /Users/hfdrake/.julia/packages/Oceananigans/thcPW/src/Simulations/run.jl:117
┌ Info:     ... initial time step complete (7.732 seconds).
└ @ Oceananigans.Simulations /Users/hfdrake/.julia/packages/Oceananigans/thcPW/src/Simulations/run.jl:126


i:     10, sim time:  0.194, wall time: 21.525 seconds, Δt:  0.0194, advective CFL: 4.60e-06, diffusive CFL: 6.30e-04
i:     20, sim time:  0.389, wall time: 21.996 seconds, Δt:  0.0194, advective CFL: 1.79e-05, diffusive CFL: 6.30e-04
i:     30, sim time:  0.583, wall time: 22.509 seconds, Δt:  0.0194, advective CFL: 3.99e-05, diffusive CFL: 6.30e-04
i:     40, sim time:  0.778, wall time: 22.963 seconds, Δt:  0.0194, advective CFL: 7.05e-05, diffusive CFL: 6.30e-04
i:     50, sim time:  0.972, wall time: 23.405 seconds, Δt:  0.0214, advective CFL: 1.21e-04, diffusive CFL: 6.93e-04
i:     60, sim time:  1.171, wall time: 23.853 seconds, Δt:  0.0214, advective CFL: 1.74e-04, diffusive CFL: 6.93e-04
i:     70, sim time:  1.385, wall time: 24.322 seconds, Δt:  0.0214, advective CFL: 2.42e-04, diffusive CFL: 6.93e-04
i:     80, sim time:  1.599, wall time: 24.874 seconds, Δt:  0.0214, advective CFL: 3.21e-04, diffusive CFL: 6.93e-04
i:     90, sim time:  1.813, wall time: 25.540 seconds, 

┌ Info: Simulation is stopping after running for 1.576 minutes.
└ @ Oceananigans.Simulations /Users/hfdrake/.julia/packages/Oceananigans/thcPW/src/Simulations/simulation.jl:202
┌ Info: Simulation time 50 seconds equals or exceeds stop time 50 seconds.
└ @ Oceananigans.Simulations /Users/hfdrake/.julia/packages/Oceananigans/thcPW/src/Simulations/simulation.jl:203


In [23]:
using NCDatasets
using CairoMakie
using Printf
using Oceananigans.Fields
using Oceananigans.AbstractOperations: volume
using NaNStatistics

In [16]:
#load simulation data to create animation


x = ds["xC"][4+1:end-4]; Nx = length(x);
#y = ds["yC"][4+1:end-4]; Ny = length(y);
z = ds["zC"][4+1:end-4]; Nz = length(z);
times = ds["time"][:];

t_final = times[end]

b_timeseries = ds["b"][4+1:end-4, 1, 4+1:end-4, :];


In [22]:
Nx

256

In [24]:
@info "Creating animation from saved data ..."
ds = NCDataset(string("/Users/hfdrake/code/HorizontalConvection/output/turbulent_h0.6_buoyancy.nc"));

times = ds["time"][:]

n = Observable(1)

x = ds["xC"][4+1:end-4]
z = ds["zC"][4+1:end-4]

H =1 
Lx = 8H

bₙ = @lift ds["b"][4+1:end-4, 1, 4+1:end-4, $n]

fig = Figure(size = (800, 300))

kwargs = (xlabel=L"x/H", 
               ylabel=L"z/H", 
               limits = ((-Lx/2, Lx/2), (-H, 0)), 
               aspect = Lx/H, 
               titlesize = 20)

title = @lift @sprintf("nondimensional buoyancy at t = %.2f", times[$n])

ax_B = Axis(fig[1,1]; title = title, kwargs...)

B_lims = (-0.6, 0.6)

b = ds["b"][4+1:end-4, 1, 4+1:end-4, 3] 
wet = b.!=0.
b[.!wet] .= NaN
wet_masked = Float64.(copy(wet))
wet_masked[wet] .= NaN

hm_b = heatmap!(ax_B, x, z, bₙ; colorrange = B_lims, colormap = :balance)
hm = heatmap!(ax_B, x, z, wet_masked[:,:], colormap = :algae)
Colorbar(fig[1,2], hm_b)

frames = 1:length(times)

record(fig, "../animations/adjustedrayleigh.mp4", frames, framerate = 8) do i 
    n[] = i 
end

close(ds)

┌ Info: Creating animation from saved data ...
└ @ Main /Users/hfdrake/code/HorizontalConvection/notebooks/VariableRayleighNum.ipynb:1


closed Dataset